# KARTOBI Sofiane 
# Implementation of a merkle tree and check inclusion (data validation)

This work is based on several videos I've watched, but here are the key ones that helped the most : 
- Bitcoin 101 - Merkle Roots and Merkle Trees : https://youtu.be/gUwXCt1qkBU?si=D9DPsnTCqKVUx79m
- The Blockchain & Bitcoin -Computerphile : https://youtu.be/qcuc3rgwZAE?si=9dvtsewFpscH0TRW
- How Merkle Trees Enable Decentralized Web : https://youtu.be/YIc6MNfv5iQ?si=P-CbTW0OceOjK28p 

These videos made me refresh my memory from the last lecture and helped dig deeper into the subject like the importance of merkle trees, how they are implemented and how it actually revolutionnzed Blockchain and Bitcoin

After watching these videos carefully first, I was able to then understand articles quickly that had source code.

I've found this article on Medium very complete and helpful but its source code in nodejs :
https://medium.com/coinmonks/merkle-tree-a-simple-explanation-and-implementation-48903442bc08 

This article is also very rich but the code was in C# : https://www.codeproject.com/Articles/1176140/Understanding-Merkle-Trees-Why-use-them-who-uses-t#HowDoesDataVerification(AuditProof)Work13



## In short here's what I've understood so far

 - Merkle tree is just a way of representing a data, so to check whether or not we have the same file (data set) we just compare the merkle root
 - The leaves are chunks of data that can come from peers in the network and they are not safe and we can't trust them
-  When it comes to partial validation, which means verifying whether or not, a chunk or set of data belongs to a certain file, or in a block chain context, we verify if a transcation belongs to a certain block, It's more efficient to verify in a merkle tree because we only look for branches that the transcations belongs to , instead of using a whole loop to go through an entire list of hashes 
- The complexity here is logarithmic and hence less than a loop, since we will need only few additional data (log(n)) to check the inclusion of data, to help us calculte the root 

![Merkle Tree](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*4Pl5Rfi6aZxgqHpJP53ZxQ.jpeg)


## Importing libraries

In [1]:
import hashlib #to use SHA-256 hash function
import collections #for creating merkle tree

## Defining Helper functions

- The "ensure_even()" function helps us make sure the chunks of data are even so that we can later on concatenate them and hash them in pairs, if not we copy the last one and hash it with itself 
- The "get_leaf_node_direction_in_merkle_tree()" function help us determine if the hash in a left or right child, which is very important as the order(direction) of hashes when concatenating can give different outcomes

In [2]:
def ensure_even(hashes):
    if len(hashes) % 2 != 0:
        hashes.append(hashes[-1])

def get_leaf_node_direction_in_merkle_tree(hash, merkle_tree):
    hash_index = merkle_tree[0].index(hash)
    return "left" if hash_index % 2 == 0 else "right"


## Define the main functions
- We use recursion to concatenate hashes and then hash the concatenation using sha256. So it first do this process on the leaf level and then gets to the top until there's only one hash left which is the merkle root

In [3]:
def generate_merkle_root(hashes):
    if not hashes or len(hashes) == 0:
        return ""

    ensure_even(hashes)

    combined_hashes = []
    for i in range(0, len(hashes), 2):
        hash_pair_concatenated = hashes[i] + hashes[i + 1]
        hash = hashlib.sha256(hash_pair_concatenated.encode()).hexdigest()
        combined_hashes.append(hash)

    if len(combined_hashes) == 1:
        return combined_hashes[0]

    return generate_merkle_root(combined_hashes)

This function works by recursively concatenating each pair of hash hashes with the correct tree branch direction (left, right) and calculating the sha256 hash of the concatenated pair, until the merkle root hash is generated and returned.

In [4]:
def get_merkle_root_from_merkle_proof(merkle_proof):
    if not merkle_proof or len(merkle_proof) == 0:
        return ""

    merkle_root_from_proof = merkle_proof[0]["hash"]
    for node in merkle_proof[1:]:
        if node["direction"] == "right":
            merkle_root_from_proof = hashlib.sha256((merkle_root_from_proof + node["hash"]).encode()).hexdigest()
        else:
            merkle_root_from_proof = hashlib.sha256((node["hash"] + merkle_root_from_proof).encode()).hexdigest()

    return merkle_root_from_proof


We start creating a tree by appending the first list of hashes which is gonna represent the leaves (level=0)
Then using recursion we will concatenate and hash pairs of hash until the list lenght is 1 which means we obtained the root

In [5]:
def generate_merkle_tree(hashes):
    if not hashes or len(hashes) == 0:
        return []
    
    tree = [hashes]

    def generate(hashes, tree):
        if len(hashes) == 1:
            return hashes

        ensure_even(hashes)

        combined_hashes = []
        for i in range(0, len(hashes), 2):
            hashes_concatenated = hashes[i] + hashes[i + 1]
            hash = hashlib.sha256(hashes_concatenated.encode()).hexdigest()
            combined_hashes.append(hash)

        tree.append(combined_hashes)
        return generate(combined_hashes, tree)

    generate(hashes, tree)
    return tree

There's a pattern : for n transactions we need  about log(n) hashes to check inclusion (Medium article)

We first generate the tree using the hashes of our transactions
Then start the merkle proof list with the hash we are checking, and we will append new hashes while looking for nodes that help us find the root based on the hash

We first start with base level (leaves) find the index of the transaction then check if it is a left or right child.
Left childs have even indexes and right childs have odd indexes
Then we add the node to the proof 
After that we look for the parent index on the next level by dividing by 2 and taking the integer part 

In [6]:
def generate_merkle_proof(hash, hashes):
    if not hash or not hashes or len(hashes) == 0:
        return None

    tree = generate_merkle_tree(hashes)

    merkle_proof = [{
        "hash": hash,
        "direction": get_leaf_node_direction_in_merkle_tree(hash, tree)
    }]

    hash_index = tree[0].index(hash)
    for level in range(0, len(tree) - 1):
        is_left_child = hash_index % 2 == 0
        sibling_direction = "right" if is_left_child else "left"
        sibling_index = hash_index + 1 if is_left_child else hash_index - 1
        sibling_node = {
            "hash": tree[level][sibling_index],
            "direction": sibling_direction
        }
        merkle_proof.append(sibling_node)
        hash_index = hash_index // 2

    return  merkle_proof


## Testing the functions on random hashes

In [7]:
hashes = [
    '95cd603fe577fa9548ec0c9b50b067566fe07c8af6acba45f6196f3a15d511f6',
    '709b55bd3da0f5a838125bd0ee20c5bfdd7caba173912d4281cae816b79a201b',
    '27ca64c092a959c7edc525ed45e845b1de6a7590d173fd2fad9133c8a779a1e3',
    '1f3cb18e896256d7d6bb8c11a6ec71f005c75de05e39beae5d93bbd1e2c8b7a9',
    '41b637cfd9eb3e2f60f734f9ca44e5c1559c6f481d49d6ed6891f3e9a086ac78',
    'a8c0cce8bb067e91cf2766c26be4e5d7cfba3d3323dc19d08a834391a1ce5acf',
    'd20a624740ce1b7e2c74659bb291f665c021d202be02d13ce27feb067eeec837',
    '281b9dba10658c86d0c3c267b82b8972b6c7b41285f60ce2054211e69dd89e15',
    'df743dd1973e1c7d46968720b931af0afa8ec5e8412f9420006b7b4fa660ba8d',
    '3e812f40cd8e4ca3a92972610409922dedf1c0dbc68394fcb1c8f188a42655e2',
    '3ebc2bd1d73e4f2f1f2af086ad724c98c8030f74c0c2be6c2d6fd538c711f35c',
    '9789f4e2339193149452c1a42cded34f7a301a13196cd8200246af7cc1e33c3b',
    'aefe99f12345aabc4aa2f000181008843c8abf57ccf394710b2c48ed38e1a66a',
    '64f662d104723a4326096ffd92954e24f2bf5c3ad374f04b10fcc735bc901a4d',
    '95a73895c9c6ee0fadb8d7da2fac25eb523fc582dc12c40ec793f0c1a70893b4',
    '315987563da5a1f3967053d445f73107ed6388270b00fb99a9aaa26c56ecba2b',
    '09caa1de14f86c5c19bf53cadc4206fd872a7bf71cda9814b590eb8c6e706fbb',
    '9d04d59d713b607c81811230645ce40afae2297f1cdc1216c45080a5c2e86a5a',
    'ab8a58ff2cf9131f9730d94b9d67f087f5d91aebc3c032b6c5b7b810c47e0132',
    'c7c3f15b67d59190a6bbe5d98d058270aee86fe1468c73e00a4e7dcc7efcd3a0',
    '27ef2eaa77544d2dd325ce93299fcddef0fae77ae72f510361fa6e5d831610b2'
]

### Generating Merkle Root

In [8]:
merkleRoot = generate_merkle_root(hashes=hashes)
merkleRoot

'68e6cdf0cae7fb8eef39cc899c8882e34dd1727a2d08f2303811886949c539e6'

### Generating Merkle Tree

In [9]:
generate_merkle_tree(hashes=hashes)

[['95cd603fe577fa9548ec0c9b50b067566fe07c8af6acba45f6196f3a15d511f6',
  '709b55bd3da0f5a838125bd0ee20c5bfdd7caba173912d4281cae816b79a201b',
  '27ca64c092a959c7edc525ed45e845b1de6a7590d173fd2fad9133c8a779a1e3',
  '1f3cb18e896256d7d6bb8c11a6ec71f005c75de05e39beae5d93bbd1e2c8b7a9',
  '41b637cfd9eb3e2f60f734f9ca44e5c1559c6f481d49d6ed6891f3e9a086ac78',
  'a8c0cce8bb067e91cf2766c26be4e5d7cfba3d3323dc19d08a834391a1ce5acf',
  'd20a624740ce1b7e2c74659bb291f665c021d202be02d13ce27feb067eeec837',
  '281b9dba10658c86d0c3c267b82b8972b6c7b41285f60ce2054211e69dd89e15',
  'df743dd1973e1c7d46968720b931af0afa8ec5e8412f9420006b7b4fa660ba8d',
  '3e812f40cd8e4ca3a92972610409922dedf1c0dbc68394fcb1c8f188a42655e2',
  '3ebc2bd1d73e4f2f1f2af086ad724c98c8030f74c0c2be6c2d6fd538c711f35c',
  '9789f4e2339193149452c1a42cded34f7a301a13196cd8200246af7cc1e33c3b',
  'aefe99f12345aabc4aa2f000181008843c8abf57ccf394710b2c48ed38e1a66a',
  '64f662d104723a4326096ffd92954e24f2bf5c3ad374f04b10fcc735bc901a4d',
  '95a73895c9c6ee0fa

### Generating Merkle Proof

In [10]:
generatedMerkleProof = generate_merkle_proof(hash=hashes[4],hashes=hashes)
generatedMerkleProof

[{'hash': '41b637cfd9eb3e2f60f734f9ca44e5c1559c6f481d49d6ed6891f3e9a086ac78',
  'direction': 'left'},
 {'hash': 'a8c0cce8bb067e91cf2766c26be4e5d7cfba3d3323dc19d08a834391a1ce5acf',
  'direction': 'right'},
 {'hash': '1013dc83aed099eafbd769ded32f5269fff327994161ecc547bf38117a5acaa0',
  'direction': 'right'},
 {'hash': 'b5fb96a64aa54c2368eb31066edb453ced97305fb795a1449a751e7d3e053317',
  'direction': 'left'},
 {'hash': '2993546ae04f8a64f0a9051932cc179bc0f7c74d8254066fb8e24fd49e9bf131',
  'direction': 'right'},
 {'hash': 'e22239034a17a55feaab2c577763a3997873b47fe893761efef923a12887d2b7',
  'direction': 'right'}]

### Checking inclusion

In [11]:
merkleRootFromMerkleProof = get_merkle_root_from_merkle_proof(generatedMerkleProof)
print("Merkle Root calculated from Merkle Proof :",merkleRootFromMerkleProof)
print(merkleRootFromMerkleProof == merkleRoot)

Merkle Root calculated from Merkle Proof : 68e6cdf0cae7fb8eef39cc899c8882e34dd1727a2d08f2303811886949c539e6
True
